In [19]:
from ohsome import OhsomeClient

import altair as alt
import altair_tiles as til

import geopandas as gpd
import pandas as pd


import folium

import branca.colormap as cm
import numpy as np

import matplotlib.pyplot as plt


from datetime import datetime


In [4]:
#This Script gets a geometry from osm via overpass and gets a osm streetnetwork incl history for this geometry.

# get rasters (w net and acc info)

In [45]:
#gdf_polygons_filtered_germany = gpd.read_file(r"output\gdf_polygons_filtered_germany.gpkg")
germany_raster2022_net_acc = gpd.read_file("../output/germany_raster2022_net_acc.gpkg")

germany_raster2022_net_acc

,id_left,2018_length_50,2024_length_50,2018_length_30,2024_length_30,2018_total_length,2018_ratio,2024_total_length,2024_ratio,diff_total_length,...,2018_count,2019_count,2020_count,2021_count,2022_count,2023_count,vorher,nachher,unfall_ratio,geometry
0,1kmN2920E4186,0.000000,328.788848,333.719777,3.172751,333.719777,1.000000,331.961600,0.009558,-1.758177,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,"MULTIPOLYGON (((8.1411 49.37758, 8.15486 49.37..."
1,1kmN3376E4317,0.000000,405.627253,429.872263,21.858073,429.872263,1.000000,427.485327,0.051132,-2.386937,...,0.0,1.0,2.0,2.0,0.0,0.0,1.0,0.0,-1.0,"MULTIPOLYGON (((9.93974 53.49176, 9.95481 53.4..."
2,1kmN3269E4561,0.000000,345.792993,372.438696,24.254439,372.438696,1.000000,370.047431,0.065544,-2.391265,...,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,-1.0,"MULTIPOLYGON (((13.53385 52.47785, 13.54855 52..."
3,1kmN2868E4278,52.623281,770.512547,734.514330,12.275386,787.137611,0.933146,782.787934,0.015682,-4.349677,...,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,"MULTIPOLYGON (((9.41341 48.92369, 9.42705 48.9..."
4,1kmN2795E4339,47.286727,974.807076,972.420970,44.923880,1019.707697,0.953627,1019.730956,0.044055,0.023259,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,"MULTIPOLYGON (((10.24235 48.26825, 10.25582 48..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7394,1kmN2732E4156,465.539668,0.000000,0.000000,510.533903,465.539668,0.000000,510.533903,1.000000,44.994236,...,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,"MULTIPOLYGON (((7.80354 47.67957, 7.81685 47.6..."
7395,1kmN3006E4203,35.760527,0.000000,0.000000,35.760527,35.760527,0.000000,35.760527,1.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,"MULTIPOLYGON (((8.34886 50.1543, 8.36285 50.15..."
7396,1kmN2763E4171,273.755348,0.000000,0.000000,278.852819,273.755348,0.000000,278.852819,1.000000,5.097470,...,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,"MULTIPOLYGON (((7.99223 47.96225, 8.00561 47.9..."
7397,1kmN2831E4201,144.273724,0.000000,0.000000,140.604107,144.273724,0.000000,140.604107,1.000000,-3.669616,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,"MULTIPOLYGON (((8.37411 48.5805, 8.38765 48.58..."


In [46]:

# doing some data cleaning, round and set order of columns

first_year = "2018"
last_year = "2024"

# List of columns to round to the nearest integer
columns_to_round = [
    f"{first_year}_length_50", f"{last_year}_length_50", f"{first_year}_length_30", f"{last_year}_length_30",
    f"{first_year}_total_length", f"{last_year}_total_length", "diff_total_length"
]

# List of columns to round to two decimal places
columns_to_round_2 = [
    f"{first_year}_ratio", f"{last_year}_ratio",
    "diff_total_length_rel", "diff_ratio_tempo30"
]

# Round the specified columns to the nearest integer and convert to integer type
germany_raster2022_net_acc[columns_to_round] = germany_raster2022_net_acc[columns_to_round].round(0).astype(int)

# Round the specified columns to two decimal places
germany_raster2022_net_acc[columns_to_round_2] = germany_raster2022_net_acc[columns_to_round_2].round(2)


# Define the new column order
new_column_order = [
    "id_left",
    f"{first_year}_length_30", f"{first_year}_length_50", f"{first_year}_total_length", f"{first_year}_ratio",
    f"{last_year}_length_30", f"{last_year}_length_50", f"{last_year}_total_length", f"{last_year}_ratio",
    "diff_total_length", "diff_total_length_rel", "diff_ratio_tempo30"
]

# Reorder the columns while keeping the rest unchanged
remaining_columns = [col for col in germany_raster2022_net_acc.columns if col not in new_column_order]
final_column_order = new_column_order + remaining_columns

# Reorder DataFrame
germany_raster2022_net_acc = germany_raster2022_net_acc[final_column_order]

gdf_rasters=germany_raster2022_net_acc.to_crs(epsg=3857).copy()


#### Prep the Folium View
* To be able to view both the network changes and the accident changes in one Folium map, I cut out a circle with a radius of 400m.
* The color of the raster shows the network changes.
* The color of the circle shows the accident changes.

In [8]:

# this will take pretty long ~ 7mins 


# Step 1: Compute the centroids of the polygons
gdf_centroids = gdf_rasters.copy()
gdf_centroids["geometry"] = gdf_rasters.geometry.centroid

# Step 2: Buffer the centroids by 400 meters
gdf_buffered_centroids = gdf_centroids.copy()
gdf_buffered_centroids["geometry"] = gdf_centroids.geometry.buffer(400)

# Step 3: Clip the original polygons by removing the buffered areas
gdf_cut = gdf_rasters.copy()
gdf_cut["geometry"] = gdf_rasters.geometry.difference(gdf_buffered_centroids.union_all())




In [ ]:

### to leave out MV, NRW and TH
bland = gpd.read_file("https://raw.githubusercontent.com/isellsoap/deutschlandGeoJSON/main/2_bundeslaender/1_sehr_hoch.geo.json")
bland_in=bland[~bland["id"].isin(["DE-MV","DE-NW","DE-TH"])]
bland_out=bland[bland["id"].isin(["DE-MV","DE-NW","DE-TH"])]

In [10]:

#count_columns = germany_raster2022_net_acc.filter(like="_count").columns.tolist()
#count_columns

In [11]:


# Ensure both GeoDataFrames are in WGS84 (EPSG:4326) for Folium
gdf_buffered_centroids = gdf_buffered_centroids.to_crs(epsg=4326)
gdf_cut = gdf_cut.to_crs(epsg=4326)

# Fix: Replace NaN values to avoid colormap errors
gdf_buffered_centroids["unfall_ratio"] = gdf_buffered_centroids["unfall_ratio"].fillna(0)
gdf_cut["diff_ratio_tempo30"] = gdf_cut["diff_ratio_tempo30"].fillna(0)

min_length = 300
min_accidents = 3

gdf_buffered_centroids=gdf_buffered_centroids[gdf_buffered_centroids[f"{last_year}_total_length"]>min_length].copy()
gdf_buffered_centroids=gdf_buffered_centroids[gdf_buffered_centroids["vorher"]>=min_accidents].copy()
gdf_cut=gdf_cut[gdf_cut[f"{last_year}_total_length"]>=min_length].copy()
gdf_cut=gdf_cut[gdf_cut["vorher"]>=min_accidents].copy()

# round the values
gdf_cut = gdf_cut.round(2)
gdf_buffered_centroids = gdf_buffered_centroids.round(2)


tooltip_fields = [
    'id_left', f'{str(first_year)}_length_30', f'{str(first_year)}_length_50', f'{str(first_year)}_total_length', f'{str(first_year)}_ratio', 
    f'{str(last_year)}_length_30', f'{str(last_year)}_length_50', f'{str(last_year)}_total_length', f'{str(last_year)}_ratio', 
    'diff_total_length', 'diff_total_length_rel', "diff_ratio_tempo30", 
    '2017_count', '2018_count', '2019_count', '2020_count', '2021_count', '2022_count', '2023_count', 
    'vorher', 'nachher', 'unfall_ratio'
]

# Create alias labels for tooltips (optional, for better readability)
tooltip_aliases = [
    "Kachel-ID", f"{first_year} Length 30", f"{first_year} Length 50", f"{first_year} Total Length", f"{first_year} Ratio", 
    f"{last_year} Length 30", f"{last_year} Length 50", f"{last_year} Total Length", f"{last_year} Ratio", 
    "Diff Total Length", "Diff Total Length Rel", "Diff Ratio Tempo30", 
    "2017 Unfall-Count", "2018 Unfall-Count", "2019 Unfall-Count", "2020 Unfall-Count", "2021 Unfall-Count", "2022 Unfall-Count", "2023 Unfall-Count", 
    "Acc Before", "Acc After", "Accident Ratio"
]

# Get max absolute values for centered colormaps
max_unfall = max(abs(gdf_buffered_centroids["unfall_ratio"].min()), abs(gdf_buffered_centroids["unfall_ratio"].max()))
max_diff_tempo30 = max(abs(gdf_cut["diff_ratio_tempo30"].min()), abs(gdf_cut["diff_ratio_tempo30"].max()))


## add the attributions
basic_attribution='\u0026copy; \u003ca href=\"http://www.openstreetmap.org/copyright\"\u003eOpenStreetMap\u003c/a\u003e contributors'
carto_attribution=basic_attribution + ' | \u0026copy; \u003ca href=\"http://cartodb.com/attributions\"\u003eCartoDB\u003c/a\u003e' # <-- note this

attr_1= ' | \u003ca href=\"https://unfallatlas.statistikportal.de/"\u003e Unfallatlas \u003c/a\u003e der Statistischen Ämter des Bundes und der Länder '
attr_2= ' | \u003ca href=\"https://plattform-npgeo-vfdb.hub.arcgis.com/datasets/esri-de-content::zensus-2022-gitterzellen/about?layer=1"\u003e Zensus2022_1kmGitter \u003c/a\u003e  © GeoBasis-DE / BKG (2024) '
btr_attr=' | Erstellt von TH Wildau \u003ca href=\"https://www.th-wildau.de/studieren-weiterbilden/studiengaenge/radverkehr-m-eng/labor-radverkehr"\u003e FG Radverkehr \u003c/a\u003e'

btr_attr=attr_1 + attr_2 + btr_attr



# Create a base map with CartoDB Positron tiles
m = folium.Map(
location=[gdf_buffered_centroids.geometry.to_crs(epsg=3857).centroid.to_crs(epsg=4326).y.mean(), 
          gdf_buffered_centroids.geometry.to_crs(epsg=3857).centroid.to_crs(epsg=4326).x.mean()], 
    zoom_start=9, 
    control_scale=True,
    tiles="CartoDB Positron",
    attr=carto_attribution + btr_attr
)


# Add bland_in as white transparent background FIRST
folium.GeoJson(
    bland_in,
    name="Bland In",
    style_function=lambda feature: {
        'fillColor': 'white',
        'color': 'white',  # White edges
        'weight': 0.5,
        'fillOpacity': 0.3  # Adjust transparency
    },
    control=False  # No need for Layer Control
).add_to(m)

# Add bland_out as gray transparent background SECOND
folium.GeoJson(
    bland_out,
    name="Bland Out",
    style_function=lambda feature: {
        'fillColor': 'gray',
        'color': 'gray',  # Gray edges
        'weight': 0.5,
        'fillOpacity': 0.3  # Adjust transparency
    },
    control=False  # No need for Layer Control
).add_to(m)



### 1. Add gdf_buffered_centroids layer with unfall_ratio coloring ###
buffer_colormap = cm.LinearColormap(['green','green','yellow', 'red', 'red'], 
                                    vmin=-1, vmax=1, 
                                    #caption="● Entwicklung Anzahl Unfälle (18&19 vs 22&23)")
                                   caption="⬤ Entwicklung Anzahl Unfälle (18&19 vs 22&23)")
def style_buffered(feature):
    return {
        'fillColor': buffer_colormap(feature['properties']['unfall_ratio']),
        'color': 'lightgray',  # Light gray edges
        'weight': 0,
        'fillOpacity': 0.7
    }

folium.GeoJson(
    gdf_buffered_centroids,
    name="Unfäll",
    style_function=style_buffered,
    #tooltip=folium.GeoJsonTooltip(fields=["unfall_ratio"], aliases=["Unfall Ratio:"])
    tooltip=folium.GeoJsonTooltip(fields=tooltip_fields, aliases=tooltip_aliases)
).add_to(m)

buffer_colormap.add_to(m)  # Add legend


### 2. Add gdf_cut layer with diff_ratio_tempo30 coloring ###
cut_colormap = cm.LinearColormap(['red', 'red', 'lightgray', 'blue', 'blue'], 
                                 vmin=-max_diff_tempo30, vmax=max_diff_tempo30, 
                                 #caption="◘ Änderung des Tempo30 ggü Tempo50 Anteils")
                                 caption="🞑 Änderung des Tempo30 ggü Tempo50 Anteils")
def style_cut(feature):
    return {
        'fillColor': cut_colormap(feature['properties']['diff_ratio_tempo30']),
        'color': 'lightgray',  # Light gray edges
        'weight': 0,
        'fillOpacity': 0.6
    }

folium.GeoJson(
    gdf_cut,
    name="Tempo",
    style_function=style_cut,
    #tooltip=folium.GeoJsonTooltip(fields=["diff_ratio_tempo30"], aliases=["Diff Ratio Tempo30:"])
    tooltip=folium.GeoJsonTooltip(fields=tooltip_fields, aliases=tooltip_aliases)
).add_to(m)

cut_colormap.add_to(m)  # Add legend

# Add Layer Control
folium.LayerControl().add_to(m)

# Save map to HTML file or display

m.save("../viz/germany_raster2022_net_acc.html")
#m  # Display in Jupyter Notebook (if applicable)
